In [2]:
import os
import csv
import sys
import random
import scipy
import numpy as np



workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

In [3]:
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

In [4]:
best_nominal_features = dict()

feat2index_read = open(workspace_path + '/dict/best_nominal_features.txt', 'r')

for line in feat2index_read:
    
    line = line.strip()
    
    item1 = line.split('|')[0]
    item2 = line.split('|')[1]
    index = int(line.split('|')[2])
    
    if item1 not in best_nominal_features:
       best_nominal_features[item1] = dict()  
    if item2 not in best_nominal_features:
       best_nominal_features[item2] = dict() 
       
    best_nominal_features[item1][item2] = index 
    best_nominal_features[item2][item1] = index
        
        

In [24]:
pos_case_dict = dict()
neg_case_dict = dict()

for i in range(1,25): pos_case_dict[i] = 0 
for i in range(1,25): neg_case_dict[i] = 0    
    
model_specif = 18

#data 5[2016July] for model 18
#data 16[2016May] for model 13
#data 12[2016Jan] for model 20
#--------------------------------------


train_write_set = open(workspace_path + '/train/model_max/cust_product_'+str(model_specif)+'.train', 'w')

check_line_count = 0


feat_data = []
for date_index_major in [5,12]:
    feat_data.append(open(workspace_path + '/L4_feat_standard/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'r'))

file_inx = 0    
for feat_file in feat_data:
    
    file_inx += 1 
    print file_inx
    
    for line in feat_file:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 6:

           print line 

        cust_id = line.split('|')[0]
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        added_product_features_lm = line.split('|')[3]
        added_product_features_le = line.split('|')[4]
        product_features = line.split('|')[5]
        nominal_features_pres = line.split('|')[6]
        real_features_pres = line.split('|')[7]
    
        #----------------------------------------------------------------------------------
        nominal_features_temp = []
        nominal_features_exp = np.zeros(10000)
        for feat_item in nominal_features_pres.split(' '):
            feat_inx_temp = feat_item.split(':')[0]
            nominal_features_temp.append(feat_inx_temp)
            
        for item1 in nominal_features_temp:
            for item2 in nominal_features_temp:
                if item1 != item2:
                   if item1 in best_nominal_features and item2 in best_nominal_features[item1]:
                      get_index = best_nominal_features[item1][item2]  
                      nominal_features_exp[get_index] = 1  
        
        nominal_features_exp_str = ''
        for inx_item in list(np.where(nominal_features_exp == 1))[0]:
            nominal_features_exp_str += str(10000+inx_item) + ':1 '
        #----------------------------------------------------------------------------------    

        get_label = np.zeros(30)

        for label_item in label_tokens.split(','):

            if len(label_item) > 0:

                prod_id = int(label_item)
                get_label[prod_id] = 1


        if True:

           for j in range(model_specif,model_specif+1):

               if get_label[j] == 1:

                    prt_line = '1' + ' '
                    #prt_line = '+1' + ' ' 
                    for fi in range(3,10):
                        if len(line.split('|')[fi]) > 0:
                           prt_line += line.split('|')[fi] + ' ' 

                        
                    #if len(nominal_features_exp_str) > 0:
                       #prt_line += nominal_features_exp_str 
                    
                        
                    train_write_set.write(prt_line[:-1]+'\n') 
                    pos_case_dict[j] += 1 

               elif get_label[j] == 0:

                    prt_line = '0' + ' '
                    #prt_line = '+1' + ' ' 
                    for fi in range(3,10):
                        if len(line.split('|')[fi]) > 0:
                           prt_line += line.split('|')[fi] + ' ' 
                        
                    #if len(nominal_features_exp_str) > 0:
                    #prt_line += nominal_features_exp_str 
                    if file_inx == 1:
                       train_write_set.write(prt_line[:-1]+'\n')  
                       neg_case_dict[j] += 1 
    
   
print check_line_count    
train_write_set.close() 

1
2
67041


In [25]:
#train_read = open(workspace_path + '/train/model_pf02/cust_product_'+str(model_no)+'.train', 'r')
#Ein_read = open(workspace_path + '../liblinear-2.1/output/cust_product_'+str(model_no)+'_inv.Ein', 'r')

model_no = 18

train_data_path = './train/model_max/cust_product_'+str(model_no)+'.train'
#train_data_path = './train/model_exp/cust_product_'+str(model_no)+'.train'
model_path = '../liblinear-2.1/model_save/cust_product_'+str(model_no)+'.model'
Ein_path = '../liblinear-2.1/output/cust_product_'+str(model_no)+'.Ein'

train_command = '../liblinear-2.1/./train -s 6 -c 2'
predi_command = '../liblinear-2.1/./predict'


#print os.system("pwd");
os.system(train_command +' '+ train_data_path +' '+ model_path);
os.system(predi_command +' '+ train_data_path +' '+ model_path +' '+ Ein_path);

